In [75]:
import pandas as pd
import numpy as np
path = '/content/drive/MyDrive/Data Lake/Ingestion/Day 0/'
p_cafef = f'{path}CafeF/'
p_cp68 = f'{path}Cổ phiếu 68/'
p_vsd = f'{path}VSD/'
p_vietstock = f'{path}Vietstock/'

p_vsd_nybs = f'{p_vsd}data_lake/niemyetbs/'
UpExchange_68 = pd.read_csv(f'{p_cp68}DataUpDownExchange/DataUpExchange/file_infor_68.csv')
p_cafe_upchange = f'{p_cafef}DataUpDownExchange/DataUpExchange/'

hny_vsd = pd.read_csv(f'{p_vsd}VSD_HuyNiemYet.csv')
hny_vietstock = pd.read_csv(f'{p_vietstock}DataUpDownExchange/DataDownExchange/Delisting.csv')

all_com = list(pd.read_csv(f'{p_vietstock}Phong /AllCompanyDone.csv')['Symbol'])

In [ ]:
hny_vietstock['Sàn GDhiện tại'] = hny_vietstock['Sàn GDhiện tại'].str.upper()
hny_vsd['Thị trường'] = hny_vsd['Thị trường'].str.upper()

# F3: lọc các kỳ theo lên sàn và xuống sàn để vào F3

In [88]:
def UpExchangeCafef(symbol):
    try:
        return pd.read_csv(f'{p_cafe_upchange}{symbol}.csv')['Value'].iloc[0]
    except: 
        return np.nan

In [89]:
UpExchangeCafef('NKD')

nan

In [ ]:
pd.read_csv(f'{p_cafe_upchange}NKD.csv')

In [ ]:
def defUpExchangeCp68(symbol, UpExchange_68):
    row_symbol = UpExchange_68[UpExchange_68['company'] == symbol]
    if len(row_symbol) != 0:
        date_gdcc = row_symbol['Ngày niêm yết'].iloc[0]
    else: date_gdcc = np.nan
    return date_gdcc
    
def UpExchangeVSD(symbol):
    try:
        return pd.read_csv(f'{p_vsd_nybs}{symbol}.csv')['Ngày cấp'].iloc[0]
    except: 
        return np.nan

In [ ]:

def date_down_exchange_vietstock(symbol, hny_vietstock):
    row_symbol = hny_vietstock[hny_vietstock['Mã CK'] == symbol]
    if len(row_symbol) != 0:
        exchange_now = row_symbol['Sàn GDhiện tại'].iloc[0]
        if exchange_now in ['HOSE', 'HNX', "UPCOM"]:
            date_gdcc = np.nan
        else:
            date_gdcc = row_symbol['Ngày GDCC▼'].iloc[0]
    else: date_gdcc = np.nan
    return date_gdcc
def date_down_exchange_vsd(symbol, hny_vsd):
    row_symbol = hny_vsd[hny_vsd['Mã chứng khoán'] == symbol]
    if len(row_symbol) != 0:
        date_gdcc = row_symbol['Ngày hủy ĐK'].iloc[0]
    else: date_gdcc = np.nan
    return date_gdcc


In [ ]:
list_vietstock_downexchange = []
list_vsd_downexchange = []
list_vsd_upexchange = []
list_cp68_upexchange = []
list_cafef_upexchange = []

for i in range(len(all_com)):
    symbol = all_com[i]
    list_vietstock_downexchange.append(date_down_exchange_vietstock(symbol, hny_vietstock))
    list_vsd_downexchange.append(date_down_exchange_vsd(symbol, hny_vsd))

    list_vsd_upexchange.append(UpExchangeVSD(symbol))
    list_cp68_upexchange.append(defUpExchangeCp68(symbol, UpExchange_68))
    
    list_cafef_upexchange.append(UpExchangeCafef(symbol))
    # print(i, symbol, date_down_exchange_vietstock(symbol, hny_vietstock), date_down_exchange_vsd(symbol, hny_vsd))

In [ ]:
pd.DataFrame({'Symbol':all_com,
              'UpExchangeVSD':list_vsd_upexchange,
              'UpExchangeCp68':list_cp68_upexchange,
              'UpExchangeCafef': list_cafef_upexchange,
              'DownExchangeVSD':list_vsd_downexchange,
              'DownExchangeVietstock':list_vietstock_downexchange,}).to_csv('/content/drive/MyDrive/Data Lake/Ingestion/Day 0/Vietstock/Phong /UpdownExchange.csv', index = False)

# check tay

In [ ]:
df_up_down_csv = pd.read_csv('/content/drive/MyDrive/Data Lake/Ingestion/Day 0/Vietstock/Phong /UpdownExchange.csv')
df_csv = df_up_down_csv.copy()
p = '/content/drive/MyDrive/Data Lake/Ingestion/Day 0/Vietstock/'
p1 = f'{p}Phong /'
df_up_down_excel= pd.read_excel(f'{p1}List_Com_DateUpDownExchange.xlsx')
df_excel = df_up_down_excel.copy()

In [ ]:

df_excel['Up_Exchange'] = df_excel['Up_Exchange'].str[-4:]
df_excel['Down_Exchange'] = df_excel['Down_Exchange'].str[-4:]
for col in df_csv.columns[1:]:
    df_csv[col] = df_csv[col].str[-4:]

In [ ]:
df = pd.merge(df_csv, df_excel)
df['1'] = df['Up_Exchange'] == df['UpExchangeVSD']
df['2'] = df['Up_Exchange'] == df['UpExchangeCp68']
df['3'] = df['Up_Exchange'] == df['UpExchangeCafef']
df['4'] = df['Down_Exchange'] == df['DownExchangeVSD']
df['5'] = df['Down_Exchange'] == df['DownExchangeVietstock']
df['UpExchangeVSD'].iloc[df[df['1'] == True].index.to_list()] = df_up_down_csv.iloc[df[df['1'] == True].index.to_list()]['UpExchangeVSD']
df['UpExchangeCp68'].iloc[df[df['2'] == True].index.to_list()] = df_up_down_csv.iloc[df[df['2'] == True].index.to_list()]['UpExchangeCp68']
df['UpExchangeCafef'].iloc[df[df['3'] == True].index.to_list()] = df_up_down_csv.iloc[df[df['3'] == True].index.to_list()]['UpExchangeCafef']
df['DownExchangeVSD'].iloc[df[df['4'] == True].index.to_list()] = df_up_down_csv.iloc[df[df['4'] == True].index.to_list()]['DownExchangeVSD']
df['DownExchangeVietstock'].iloc[df[df['5'] == True].index.to_list()] = df_up_down_csv.iloc[df[df['5'] == True].index.to_list()]['DownExchangeVietstock']

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
pd.merge(df_up_down_excel, df,how = 'left', on = ['Symbol']).to_excel('hi.xlsx', index = False)

#All_com_final_have_data

In [ ]:
all_com_dontremove = pd.read_csv('/content/drive/MyDrive/06 2022/Data/Data_final/AllCompany_dontremove.csv')
all_company_done = pd.read_csv('/content/drive/MyDrive/06 2022/Data/Data_final/AllCompanyDone.csv')[['Symbol']]
pd.merge(all_company_done, all_com_dontremove, how = 'left', on = ['Symbol']).to_excel('/content/drive/MyDrive/06 2022/Data/Data_final/AllCompanyDoneCheck.xlsx', index = False)